# Phase 3: Cross-Encoder Reranking


## Step 0 — Check GPU + Install Dependencies

In [ ]:
# Verify GPU is available
import torch
print(f'CUDA available : {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU            : {torch.cuda.get_device_name(0)}')
else:
    print('⚠️  No GPU found. Go to Runtime → Change runtime type → T4 GPU')

!pip install sentence-transformers faiss-gpu pandas numpy tqdm -q
print('✅ Dependencies installed.')

## Step 1 — Mount Google Drive

In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

DATA_DIR = Path('/content/drive/MyDrive/ShopSense/data')

# Verify all required files are present
required = ['products_clean.csv', 'faiss_index.bin', 'faiss_metadata.json']
print('Checking required files...')
for fname in required:
    fpath = DATA_DIR / fname
    if fpath.exists():
        print(f'  ✅ {fname} ({fpath.stat().st_size/1024:.0f} KB)')
    else:
        print(f'  ❌ {fname} NOT FOUND — upload it to Drive first!')

## Step 2 — Load FAISS Index + Metadata

In [ ]:
import faiss
import json
import pandas as pd
import numpy as np

# Load FAISS index
index = faiss.read_index(str(DATA_DIR / 'faiss_index.bin'))
print(f'✅ FAISS index loaded — {index.ntotal:,} vectors')

# Load metadata
with open(DATA_DIR / 'faiss_metadata.json') as f:
    metadata = json.load(f)
print(f'✅ Metadata loaded   — {len(metadata):,} products')

# Load products dataframe (used for price filtering later)
df = pd.read_csv(DATA_DIR / 'products_clean.csv')

# Clean up NaN prices — fill with median price
median_price = df['price'].median()
df['price'] = df['price'].fillna(median_price)

# Also update metadata prices to match
price_map = df.set_index('asin')['price'].to_dict()
for item in metadata:
    if item.get('price') is None or str(item.get('price')) == 'nan':
        item['price'] = price_map.get(item['asin'], median_price)

print(f'✅ Products loaded   — {len(df):,} rows')
print(f'   Median price used for NaN fill: ${median_price:.2f}')

## Step 3 — Load Bi-Encoder (same model as Phase 2)
This embeds the query into a vector for FAISS search.

In [ ]:
from sentence_transformers import SentenceTransformer

print('Loading bi-encoder (same as Phase 2)...')
bi_encoder = SentenceTransformer('all-MiniLM-L6-v2')
bi_encoder.max_seq_length = 256

print(f'✅ Bi-encoder loaded')
print(f'   Model : all-MiniLM-L6-v2')
print(f'   Dim   : {bi_encoder.get_sentence_embedding_dimension()}')

## Step 4 — Load Cross-Encoder Reranker
`cross-encoder/ms-marco-MiniLM-L-6-v2` is trained on MS MARCO — a massive dataset of real search queries and passage relevance judgements. It scores (query, product) pairs directly.

In [ ]:
from sentence_transformers import CrossEncoder

print('Loading cross-encoder reranker...')
cross_encoder = CrossEncoder(
    'cross-encoder/ms-marco-MiniLM-L-6-v2',
    max_length=512,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

print(f'✅ Cross-encoder loaded')
print(f'   Model  : cross-encoder/ms-marco-MiniLM-L-6-v2')
print(f'   Device : {"GPU (cuda)" if torch.cuda.is_available() else "CPU"}')

## Step 5 — Build the Full Retrieve + Rerank Pipeline

In [ ]:
import time

def retrieve_and_rerank(
    query: str,
    top_k_retrieve: int = 20,   # FAISS retrieves this many
    top_k_final: int = 5,       # reranker picks this many
    min_price: float = None,    # optional price filter
    max_price: float = None,    # optional price filter
    min_rating: float = None    # optional rating filter
) -> dict:
    """
    Full pipeline: FAISS retrieval → optional filtering → cross-encoder reranking.

    Returns dict with results + timing info.
    """
    t0 = time.time()

    # ── Stage 1: FAISS Retrieval ──────────────────────────────
    query_embedding = bi_encoder.encode(
        [query],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype('float32')

    scores, indices = index.search(query_embedding, top_k_retrieve)

    candidates = []
    for score, idx in zip(scores[0], indices[0]):
        if idx == -1:
            continue
        product = metadata[idx].copy()
        product['faiss_score'] = round(float(score), 4)
        candidates.append(product)

    t1 = time.time()

    # ── Stage 2: Optional Filters ─────────────────────────────
    filtered = candidates
    if min_price is not None:
        filtered = [p for p in filtered if p.get('price') and p['price'] >= min_price]
    if max_price is not None:
        filtered = [p for p in filtered if p.get('price') and p['price'] <= max_price]
    if min_rating is not None:
        filtered = [p for p in filtered if p.get('rating') and p['rating'] >= min_rating]

    # If filters removed too many, fall back to unfiltered
    if len(filtered) < 3:
        print(f'  ⚠️  Only {len(filtered)} candidates after filtering — relaxing filters')
        filtered = candidates

    t2 = time.time()

    # ── Stage 3: Cross-Encoder Reranking ──────────────────────
    # Build (query, product_text) pairs for the cross-encoder
    pairs = [
        (query, f"{p['title']}. {str(p.get('description', ''))[:200]}")
        for p in filtered
    ]

    rerank_scores = cross_encoder.predict(pairs)

    # Attach rerank scores and sort
    for product, score in zip(filtered, rerank_scores):
        product['rerank_score'] = round(float(score), 4)

    reranked = sorted(filtered, key=lambda x: x['rerank_score'], reverse=True)
    final = reranked[:top_k_final]

    t3 = time.time()

    return {
        'query'        : query,
        'results'      : final,
        'timing': {
            'faiss_ms'  : round((t1 - t0) * 1000, 1),
            'filter_ms' : round((t2 - t1) * 1000, 1),
            'rerank_ms' : round((t3 - t2) * 1000, 1),
            'total_ms'  : round((t3 - t0) * 1000, 1)
        },
        'counts': {
            'retrieved' : len(candidates),
            'after_filter': len(filtered),
            'final'     : len(final)
        }
    }

print('✅ retrieve_and_rerank() pipeline ready.')

## Step 6 — Display Helper

In [ ]:
def display_reranked(result: dict, show_scores: bool = True):
    """Pretty print reranked results with before/after score comparison."""
    print(f'🔍 Query : "{result["query"]}"')
    print(f'⏱️  Timing: FAISS {result["timing"]["faiss_ms"]}ms '
          f'| Rerank {result["timing"]["rerank_ms"]}ms '
          f'| Total {result["timing"]["total_ms"]}ms')
    print(f'📦 Retrieved {result["counts"]["retrieved"]} → '
          f'Filtered to {result["counts"]["after_filter"]} → '
          f'Final {result["counts"]["final"]}')
    print('─' * 68)
    for i, r in enumerate(result['results'], 1):
        price_str = f"${r['price']:.2f}" if r.get('price') else 'N/A'
        print(f"{i}. {r['title'][:70]}")
        print(f"   Price: {price_str}  |  Rating: {r.get('rating', 'N/A')}⭐ "
              f"({int(r.get('rating_count', 0)):,} reviews)")
        if show_scores:
            print(f"   FAISS score: {r['faiss_score']}  →  Rerank score: {r['rerank_score']}")
        print()
    print('=' * 68)

print('✅ Display helper ready.')

## Step 7 — Test the Pipeline
Same queries as Phase 2 so you can directly compare before/after reranking.

In [ ]:
# Query 1 — Phase 2 returned a coffee TABLE as top result. Let's see if reranker fixes it.
result1 = retrieve_and_rerank('compact coffee maker for small kitchen')
display_reranked(result1)

In [ ]:
# Query 2 — with price filter this time
result2 = retrieve_and_rerank('non stick frying pan', max_price=30.0)
display_reranked(result2)

In [ ]:
# Query 3 — gift query
result3 = retrieve_and_rerank('gift for someone who loves cooking')
display_reranked(result3)

In [ ]:
# Query 4 — storage with rating filter
result4 = retrieve_and_rerank('storage solution for small apartment', min_rating=4.3)
display_reranked(result4)

In [ ]:
# Query 5 — try your own!
result5 = retrieve_and_rerank('durable water bottle for gym')
display_reranked(result5)

## Step 8 — Before vs After Reranking Comparison
Shows exactly how reranking changed the order from raw FAISS results.

In [ ]:
def compare_before_after(query: str, top_k_retrieve: int = 20, top_k_final: int = 5):
    """Side by side comparison of FAISS-only vs reranked results."""

    # Get FAISS only results
    query_embedding = bi_encoder.encode(
        [query], normalize_embeddings=True, convert_to_numpy=True
    ).astype('float32')
    scores, indices = index.search(query_embedding, top_k_retrieve)

    faiss_results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx != -1:
            p = metadata[idx].copy()
            p['faiss_score'] = round(float(score), 4)
            faiss_results.append(p)

    # Get reranked results
    reranked = retrieve_and_rerank(query, top_k_retrieve, top_k_final)

    print(f'🔍 Query: "{query}"')
    print(f'{"FAISS Only (top 5)":<45} {"After Reranking (top 5)"}')
    print('─' * 100)
    for i in range(top_k_final):
        left  = faiss_results[i]['title'][:42] if i < len(faiss_results) else ''
        right = reranked['results'][i]['title'][:42] if i < len(reranked['results']) else ''
        changed = '🔄' if left != right else '  '
        print(f"{i+1}. {left:<45} {changed}  {i+1}. {right}")
    print()

# Run comparison on the coffee query where Phase 2 had the wrong top result
compare_before_after('compact coffee maker for small kitchen')

In [ ]:
compare_before_after('gift for someone who loves cooking')

## Step 9 — Save Pipeline Results to Drive
Saves sample query results as JSON — useful for Phase 7 evaluation.

In [ ]:
sample_queries = [
    'compact coffee maker for small kitchen',
    'non stick frying pan',
    'gift for someone who loves cooking',
    'storage solution for small apartment',
    'durable water bottle for gym',
    'budget friendly knife set for beginner cook',
    'air fryer for family of four',
    'cute kitchen decor for modern home'
]

all_results = []
print('Running all sample queries...')
for q in sample_queries:
    r = retrieve_and_rerank(q)
    all_results.append(r)
    print(f'  ✅ "{q}"')

# Save to Drive
results_path = DATA_DIR / 'sample_reranked_results.json'
with open(results_path, 'w') as f:
    json.dump(all_results, f, indent=2)

print(f'\n✅ Saved {len(all_results)} query results → {results_path}')

## Step 10 — Final Summary

In [ ]:
# Show average timing across all sample queries
avg_faiss  = sum(r['timing']['faiss_ms']  for r in all_results) / len(all_results)
avg_rerank = sum(r['timing']['rerank_ms'] for r in all_results) / len(all_results)
avg_total  = sum(r['timing']['total_ms']  for r in all_results) / len(all_results)

print('=' * 55)
print('     PHASE 3 COMPLETE — SUMMARY')
print('=' * 55)
print(f'  Bi-encoder    : all-MiniLM-L6-v2')
print(f'  Cross-encoder : cross-encoder/ms-marco-MiniLM-L-6-v2')
print(f'  Pipeline      : FAISS top-20 → rerank → top-5')
print()
print(f'  Average Latency across {len(all_results)} queries:')
print(f'    FAISS retrieval : {avg_faiss:.1f} ms')
print(f'    Reranking       : {avg_rerank:.1f} ms')
print(f'    Total           : {avg_total:.1f} ms')
print()
print('  Files saved to Drive:')
print('    ✅ data/sample_reranked_results.json')
print()
print('  ➡️  Next: Phase 4 — RAG Review Summarization (VS Code)')
print('=' * 55)